In [1]:
!python -m pip install requests
!python -m pip install imdbpy
!python -m pip install pandas
!python -m pip install tmdbsimple
!python -m pip install psycopg2
!python -m pip install dask

In [2]:
# For debug in Colab
# from google.colab import drive
# drive.mount('/content/drive')w

## Data Collection and Preparation for existing datasets

#### Preprocess the larger kaggle datasets 
- Sources from: https://www.kaggle.com/datasets/akshaypawar7/millions-of-moviesw
- This will only run one time to get the information from the dataset:
We have already obtained the file in the moive_data table, so no need to rerun this

In [3]:
# import dask.dataframe as dd
# import tmdbsimple as tmdb
# import requests
# # Set up TMDB API credentials
# tmdb.API_KEY = '9b023caa1c3becd6d577ab0e41927545'

# # Define function to get director information from TMDB API
# def get_director_info(movie_id):
#     url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={tmdb.API_KEY}"
#     response = requests.get(url)
#     try:
#         response.raise_for_status()
#         movie_info = response.json()
#         if 'crew' in movie_info:
#             for crew_member in movie_info['crew']:
#                 if crew_member['job'] == 'Director':
#                     return {'id': crew_member['id'], 'name': crew_member['name']}
#     except requests.exceptions.HTTPError as e:
#         if e.response.status_code == 404:
#             print(f"Movie with ID {movie_id} not found.")
#         else:
#             raise e
#     return None

# # Define a function to process a chunk of the movies DataFrame
# def process_chunk(chunk):
#     # Keep only relevant columns
#     chunk = chunk.loc[:, ['id', 'title', 'release_date', 'budget', 'revenue', 'credits']]
#     # Get director information for each movie in the chunk
#     directors = []
#     for index, row in chunk.iterrows():
#         movie_id = row['id']
#         director = get_director_info(movie_id)
#         if director is not None:
#             directors.append(director['name'])
#         else:
#             directors.append(None)
#     # Add director column to the chunk
#     chunk['director'] = directors
#     return chunk

# # Read movies.csv into a dask dataframe in chunks, starting from row 350000
# chunk_size = 10000
# start_row = 0
# #Change the location to your file location
# df = dd.read_csv('/content/drive/MyDrive/movies.csv', blocksize=None, assume_missing=True,
#                 skiprows=range(1, start_row+1), usecols=['id', 'title', 'release_date', 'budget', 'revenue', 'credits'])

# # Use dask.delayed to process the chunks
# output_file = '/content/drive/MyDrive/movies_with_directors.csv'
# chunks = df.to_delayed()
# outputs = []
# for i, chunk in enumerate(chunks):
#     processed_chunk = dask.delayed(process_chunk)(chunk)
#     outputs.append(processed_chunk)
#     print(f"Submitted chunk {i}")
# # Write the processed chunks to the output CSV file as they complete
# results = dask.compute(*outputs)
# with open(output_file, 'w') as f:
#     first = True
#     for chunk in results:
#         chunk.to_csv(f, mode='a', header=first, index=False)
#         if first:
#             first = False



#### Preprocess the smaller kaggle datasets 
- source from：https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset 

In [4]:
import pandas as pd
import ast

# Load the two datasets from the old kaggle

# # Load the two datasets from the colab
# movies_file = "/content/drive/MyDrive/movies_metadata.csv"
# credits_file = "/content/drive/MyDrive/credits.csv"

movies_file = "movie_data/movies_metadata.csv"
credits_file = "movie_data/credits.csv"

df_movies = pd.read_csv(movies_file, dtype={'id': 'object'})
df_credits = pd.read_csv(credits_file, dtype={'id': 'object', 'cast': 'object', 'crew': 'str'})

# Merge the two datasets on "id"
df_merged = pd.merge(df_movies, df_credits, on='id')

# Define a function to extract the directors
def get_directors(crew_list):
    directors = []
    for crew_member in ast.literal_eval(crew_list):
        if crew_member['job'] == 'Director':
            directors.append(crew_member['name'])
    return directors

# Define a function to extract the cast
def get_cast(cast_list):
    cast = []
    for cast_member in ast.literal_eval(cast_list):
        cast.append(cast_member['name'])
    return cast

# Apply the functions to extract the directors and cast
df_merged['director'] = df_merged['crew'].apply(get_directors)
df_merged['cast'] = df_merged['cast'].apply(get_cast)
df_merged = df_merged.drop('crew', axis=1)

C:\Users\hongj\AppData\Local\Temp\ipykernel_13884\3351125354.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(movies_file, dtype={'id': 'object'})


In [5]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45538 entries, 0 to 45537
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45538 non-null  object 
 1   belongs_to_collection  4500 non-null   object 
 2   budget                 45538 non-null  object 
 3   genres                 45538 non-null  object 
 4   homepage               7792 non-null   object 
 5   id                     45538 non-null  object 
 6   imdb_id                45521 non-null  object 
 7   original_language      45527 non-null  object 
 8   original_title         45538 non-null  object 
 9   overview               44584 non-null  object 
 10  popularity             45535 non-null  object 
 11  poster_path            45152 non-null  object 
 12  production_companies   45535 non-null  object 
 13  production_countries   45535 non-null  object 
 14  release_date           45451 non-null  object 
 15  re

#### Clean up the larger kaggle dataset to be merged later

In [6]:
import pandas as pd
# # if from colab
# df_movies_tmdb = pd.read_csv("/content/drive/MyDrive/movies_with_directors.csv", dtype={'id': 'object'})
df_movies_tmdb = pd.read_csv("movie_data/movies_with_directors.csv", dtype={'id': 'object'})
df_movies_tmdb = df_movies_tmdb[['id', 'title', 'release_date', 'budget', 'revenue', 'credits', 'director']]
df_movies_tmdb = df_movies_tmdb.rename(columns={
    'title': 'original_title',
    'credits': 'cast'
})
# # from colab
# more_movies_file = "/content/drive/MyDrive/movies.csv"
more_movies_file = "movie_data/movies.csv"
df_movies_larger = pd.read_csv(more_movies_file, dtype={'id': 'object'})

# Merge the 'director' column from df_movies_tmdb into df_movies_larger based on 'id'
df_movies_larger = df_movies_larger.merge(df_movies_tmdb[['id', 'director']], on='id', how='left')
df_movies_larger = df_movies_larger.rename(columns={
    'title': 'original_title',
    'credits': 'cast'
})
# Print the updated dataframe
df_movies_larger


,id,original_title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,...,status,tagline,vote_average,vote_count,cast,keywords,poster_path,backdrop_path,recommendations,director
0,677179,Creed III,Drama-Action,en,After dominating the boxing world Adonis Creed...,9575.225,Metro-Goldwyn-Mayer-Proximity Media-Balboa Pro...,2023-03-01,75000000.0,2.580000e+08,...,Released,You can't run from your past.,7.305,845.0,Michael B. Jordan-Tessa Thompson-Jonathan Majo...,philadelphia pennsylvania-husband wife relatio...,/vJU3rXSP9hwUuLeq8IpfsJShLOk.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-107...,Michael B. Jordan
1,76600,Avatar: The Way of Water,Science Fiction-Adventure-Action,en,Set more than a decade after the events of the...,9366.788,20th Century Studios-Lightstorm Entertainment,2022-12-14,350000000.0,2.312336e+09,...,Released,Return to Pandora.,7.751,6748.0,Sam Worthington-Zoe Saldaña-Sigourney Weaver-S...,loss of loved one-dying and death-alien life-f...,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/s16H6tpK2utvwDtzZ8Qy4qm5Emw.jpg,183392-111332-702432-505642-1064215-436270-874...,James Cameron
2,502356,The Super Mario Bros. Movie,Animation-Adventure-Family-Fantasy-Comedy,en,While working underground to fix a water main ...,5132.098,Universal Pictures-Illumination-Nintendo,2023-04-05,100000000.0,5.800000e+07,...,Released,NaN,7.556,332.0,Chris Pratt-Anya Taylor-Joy-Charlie Day-Jack B...,video game-plumber-magic mushroom-based on vid...,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,/iw0Na1UBHgA5BgifwmQ8vKhlWgA.jpg,NaN,Michael Jelenic
3,631842,Knock at the Cabin,Horror-Mystery-Thriller,en,While vacationing at a remote cabin a young gi...,3422.537,Blinding Edge Pictures-Universal Pictures-Film...,2023-02-01,20000000.0,5.200000e+07,...,Released,Save your family or save humanity. Make the ch...,6.457,888.0,Dave Bautista-Jonathan Groff-Ben Aldridge-Kris...,based on novel or book-sacrifice-cabin-faith-e...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg,/zWDMQX0sPaW2u0N2pJaYA8bVVaJ.jpg,1058949-646389-772515-505642-143970-667216-104...,M. Night Shyamalan
4,646389,Plane,Action-Adventure-Thriller,en,After a heroic job of successfully landing his...,2618.646,MadRiver Pictures-Di Bonaventura Pictures-G-BA...,2023-01-12,25000000.0,5.100000e+07,...,Released,Survive together or die alone.,6.901,785.0,Gerard Butler-Mike Colter-Yoson An-Tony Goldwy...,pilot-airplane-philippines-held hostage-plane ...,/qi9r5xBgcc9KTxlOLjssEbDgO0J.jpg,/9Rq14Eyrf7Tu1xk0Pl7VcNbNh1n.jpg,505642-758769-864692-631842-1058949-925943-758...,Jean-François Richet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855565,887860,拯救校花计划,Drama,zh,NaN,0.600,NaN,2017-12-02,0.0,0.000000e+00,...,Released,NaN,0.000,0.0,NaN,NaN,/2LXG2Zhp2diqM0qLNzhZ4zbQvof.jpg,/cE4gpxY3OASBGPpv0fQqHWL4dMd.jpg,NaN,NaN
855566,886714,Oberhausen 34. Kurzfilmtage,NaN,de,01.04.1988,0.600,NaN,1988-04-01,0.0,0.000000e+00,...,Released,NaN,0.000,0.0,NaN,NaN,/yevFfrQxkwJwMMhEJqDrJKgXFp.jpg,NaN,NaN,Schmelzdahin
855567,774222,Hader fürs Heim,NaN,de,Josef Hader's first television broadcast a mix...,0.600,NaN,1991-01-01,0.0,0.000000e+00,...,Released,NaN,0.000,0.0,Josef Hader-Maria Hofstätter-Alfred Dorfer,NaN,/hBupAGQkT1yPdwZUXcxksxUsGkr.jpg,NaN,NaN,NaN
855568,832174,Into The Light : A Trip Through Baltimore Ball...,NaN,en,A not-so-Cinderella-story of LGBTQ+ Baltimore ...,0.600,NaN,2021-05-21,0.0,0.000000e+00,...,Released,NaN,0.000,0.0,Keith Holt-Londyn Smith de Richelieu-Marquis C...,NaN,/h3gUEQ2CLMdbv95l5Sxd2VDG0Bx.jpg,NaN,NaN,Jason Gray


In [7]:
df_movies_larger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855570 entries, 0 to 855569
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    855570 non-null  object 
 1   original_title        855564 non-null  object 
 2   genres                620773 non-null  object 
 3   original_language     855570 non-null  object 
 4   overview              721075 non-null  object 
 5   popularity            855570 non-null  float64
 6   production_companies  411577 non-null  object 
 7   release_date          799735 non-null  object 
 8   budget                855570 non-null  float64
 9   revenue               855570 non-null  float64
 10  runtime               816628 non-null  float64
 11  status                855570 non-null  object 
 12  tagline               131167 non-null  object 
 13  vote_average          855570 non-null  float64
 14  vote_count            855570 non-null  float64
 15  

#### Merge the dataset and clean up the dataset

In [8]:
merged_df = pd.merge(df_merged, df_movies_larger, on='id', how='outer')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859677 entries, 0 to 859676
Data columns (total 46 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   adult                   51657 non-null   object 
 1   belongs_to_collection   4626 non-null    object 
 2   budget_x                51657 non-null   object 
 3   genres_x                51657 non-null   object 
 4   homepage                9324 non-null    object 
 5   id                      859677 non-null  object 
 6   imdb_id                 51640 non-null   object 
 7   original_language_x     51643 non-null   object 
 8   original_title_x        51657 non-null   object 
 9   overview_x              50483 non-null   object 
 10  popularity_x            51654 non-null   object 
 11  poster_path_x           51161 non-null   object 
 12  production_companies_x  51654 non-null   object 
 13  production_countries    51654 non-null   object 
 14  release_date_x      

In [9]:
columns_to_merge = ['belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language',
                    'original_title', 'overview', 'popularity', 'poster_path', 'production_companies',
                    'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
                    'tagline', 'video', 'vote_average', 'vote_count', 'cast', 'director']

for column in columns_to_merge:
    x_column = f"{column}_x"
    y_column = f"{column}_y"

    if x_column in merged_df.columns and y_column in merged_df.columns:
        merged_df[column] = merged_df[y_column].combine_first(merged_df[x_column])
        merged_df = merged_df.drop(columns=[x_column, y_column])
    elif x_column in merged_df.columns:
        merged_df = merged_df.rename(columns={x_column: column})
    elif y_column in merged_df.columns:
        merged_df = merged_df.rename(columns={y_column: column})
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859677 entries, 0 to 859676
Data columns (total 29 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   adult                  51657 non-null   object 
 1   belongs_to_collection  4626 non-null    object 
 2   homepage               9324 non-null    object 
 3   id                     859677 non-null  object 
 4   imdb_id                51640 non-null   object 
 5   production_countries   51654 non-null   object 
 6   spoken_languages       51654 non-null   object 
 7   title                  51654 non-null   object 
 8   video                  51654 non-null   object 
 9   keywords               255914 non-null  object 
 10  backdrop_path          277217 non-null  object 
 11  recommendations        40532 non-null   object 
 12  budget                 859677 non-null  object 
 13  genres                 625538 non-null  object 
 14  original_language      859677 non-nu

In [10]:
merged_df

,adult,belongs_to_collection,homepage,id,imdb_id,production_countries,spoken_languages,title,video,keywords,...,production_companies,release_date,revenue,runtime,status,tagline,vote_average,vote_count,cast,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",http://toystory.disney.com/toy-story,862,tt0114709,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,False,martial arts-jealousy-friendship-bullying-elem...,...,Pixar,1995-10-30,373554033.0,81.0,Released,NaN,7.966,16430.0,Tom Hanks-Tim Allen-Don Rickles-Jim Varney-Wal...,John Lasseter
1,False,NaN,NaN,8844,tt0113497,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,False,giant insect-board game-jungle-disappearance-r...,...,TriStar Pictures-PolyGram Filmed Entertainment...,1995-12-15,262821940.0,104.0,Released,Roll the dice and unleash the excitement!,7.237,9437.0,Robin Williams-Kirsten Dunst-Bradley Pierce-Bo...,Joe Johnston
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,15602,tt0113228,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,False,fishing-halloween-sequel-old man-best friend-w...,...,Lancaster Gate-Warner Bros. Pictures,1995-12-22,71500000.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,6.500,324.0,Walter Matthau-Jack Lemmon-Ann-Margret-Sophia ...,Howard Deutch
3,False,NaN,NaN,31357,tt0114885,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,False,based on novel or book-interracial relationshi...,...,20th Century Fox,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,6.300,128.0,Whitney Houston-Angela Bassett-Loretta Devine-...,Forest Whitaker
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",NaN,11862,tt0113041,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,False,parent child relationship-baby-midlife crisis-...,...,Sandollar Productions-Touchstone Pictures,1995-12-08,76594107.0,106.0,Released,Just When His World Is Back To Normal... He's ...,6.200,626.0,Steve Martin-Diane Keaton-Martin Short-Kimberl...,Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859672,NaN,NaN,NaN,887860,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-12-02,0.0,0.0,Released,NaN,0.000,0.0,NaN,NaN
859673,NaN,NaN,NaN,886714,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1988-04-01,0.0,3.0,Released,NaN,0.000,0.0,NaN,Schmelzdahin
859674,NaN,NaN,NaN,774222,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1991-01-01,0.0,40.0,Released,NaN,0.000,0.0,Josef Hader-Maria Hofstätter-Alfred Dorfer,NaN
859675,NaN,NaN,NaN,832174,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2021-05-21,0.0,25.0,Released,NaN,0.000,0.0,Keith Holt-Londyn Smith de Richelieu-Marquis C...,Jason Gray


In [11]:
# Filter out invalid box office rows
merged_df_with_valid_box_office = merged_df[(merged_df['budget'].notnull()) & (merged_df['revenue'] >= 100)]
merged_df_with_valid_box_office


,adult,belongs_to_collection,homepage,id,imdb_id,production_countries,spoken_languages,title,video,keywords,...,production_companies,release_date,revenue,runtime,status,tagline,vote_average,vote_count,cast,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",http://toystory.disney.com/toy-story,862,tt0114709,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,False,martial arts-jealousy-friendship-bullying-elem...,...,Pixar,1995-10-30,373554033.0,81.0,Released,NaN,7.966,16430.0,Tom Hanks-Tim Allen-Don Rickles-Jim Varney-Wal...,John Lasseter
1,False,NaN,NaN,8844,tt0113497,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,False,giant insect-board game-jungle-disappearance-r...,...,TriStar Pictures-PolyGram Filmed Entertainment...,1995-12-15,262821940.0,104.0,Released,Roll the dice and unleash the excitement!,7.237,9437.0,Robin Williams-Kirsten Dunst-Bradley Pierce-Bo...,Joe Johnston
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,15602,tt0113228,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,False,fishing-halloween-sequel-old man-best friend-w...,...,Lancaster Gate-Warner Bros. Pictures,1995-12-22,71500000.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,6.500,324.0,Walter Matthau-Jack Lemmon-Ann-Margret-Sophia ...,Howard Deutch
3,False,NaN,NaN,31357,tt0114885,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,False,based on novel or book-interracial relationshi...,...,20th Century Fox,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,6.300,128.0,Whitney Houston-Angela Bassett-Loretta Devine-...,Forest Whitaker
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",NaN,11862,tt0113041,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,False,parent child relationship-baby-midlife crisis-...,...,Sandollar Productions-Touchstone Pictures,1995-12-08,76594107.0,106.0,Released,Just When His World Is Back To Normal... He's ...,6.200,626.0,Steve Martin-Diane Keaton-Martin Short-Kimberl...,Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857900,NaN,NaN,NaN,791261,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,50000000.0,124.0,Released,Boring.,0.000,0.0,NaN,NaN
858502,NaN,NaN,NaN,783065,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,150.0,0.0,Released,NaN,0.000,0.0,NaN,NaN
858622,NaN,NaN,NaN,757237,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2018-02-13,500.0,0.0,Released,Russell and Sarah are faced with what to do af...,0.000,0.0,NaN,NaN
858829,NaN,NaN,NaN,830358,NaN,NaN,NaN,NaN,NaN,trauma-turkey-based on true story-nature run a...,...,NaN,2016-04-01,720.0,2.0,Released,"« Je criais: ""Y'a une bibitte noire dans le sa...",10.000,1.0,NaN,NaN


## Data Collection and Preparation for new data

### Getting new movie information from the Movie Database (TMDb) API

In [12]:
import requests
import pandas as pd
from pandas import json_normalize

tmdb_api_key = '9b023caa1c3becd6d577ab0e41927545'

def get_movie_info(movie_id):
    # Make a request to the TMDb API for the movie information
   # https://api.themoviedb.org/3/movie/502356?api_key=9b023caa1c3becd6d577ab0e41927545&language=en-US 
    url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}&language=en-US'
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Retrieve the relevant information from the response JSON
        movie_info = response.json()
        return movie_info
    else:
        print(f"Request failed with status code {response.status_code}")

# Testing
movie_id = '760348' 
movie_dict = get_movie_info(movie_id)
flat_movie_dict = json_normalize(movie_dict)
movies_df = pd.DataFrame(flat_movie_dict)
movies_df

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/wrvnGcLxmGTlgnZyQyKYaTnibdh.jpg,None,500,"[{'id': 10402, 'name': 'Music'}]",http://harjaspreetsingh.com/projects/gal-dil-di/,760348,tt13394078,pa,Gal Dil Di,...,2020-11-03,700,4,"[{'english_name': 'Punjabi', 'iso_639_1': 'pa'...",Released,,Gal Dil Di,True,0.0,0


In [13]:
# Retrieve the newest movies that being released

def get_new_movies(num_movies_to_get):
    BASE_URL = 'https://api.themoviedb.org/3/movie/now_playing'
    movies = []
    page = 1

    while len(movies) < num_movies_to_get:
        response = requests.get(BASE_URL, params={'api_key': tmdb_api_key, 'page': page})
        if response.status_code == 200:
            data = response.json()
            for movie in data['results']:
                movie_info = get_movie_info(movie['id'])
                if movie_info is not None:
                    movies.append(movie_info)
                if len(movies) >= num_movies_to_get:
                    break
            page += 1
        else:
            break

    return movies

new_movies_df = pd.DataFrame(get_new_movies(50))
new_movies_df

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,None,18000000,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",https://www.thepopes-exorcist.movie,758323,tt13375076,en,The Pope's Exorcist,...,2023-04-05,65675816,103,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Inspired by the actual files of Father Gabriel...,The Pope's Exorcist,False,7.409,640
1,False,/iJQIbOPm81fPEGKt5BPuZmfnA54.jpg,None,100000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",https://www.thesupermariobros.movie,502356,tt6718170,en,The Super Mario Bros. Movie,...,2023-04-05,1121048165,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,The Super Mario Bros. Movie,False,7.498,1867
2,False,/nDxJJyA5giRhXx96q1sWbOUjMBI.jpg,"{'id': 724848, 'name': 'Shazam! Collection', '...",125000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",,594767,tt10151854,en,Shazam! Fury of the Gods,...,2023-03-15,133437105,130,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Oh. My. Gods.,Shazam! Fury of the Gods,False,6.784,1513
3,False,/A7JQ7MIV5fkIxceI5hizRIe6DRJ.jpg,"{'id': 284433, 'name': 'Guardians of the Galax...",250000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/guardians-of-the...,447365,tt6791350,en,Guardians of the Galaxy Volume 3,...,2023-05-03,289312702,150,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Once more with feeling.,Guardians of the Galaxy Volume 3,False,8.300,710
4,False,/44immBwzhDVyjn87b3x3l9mlhAD.jpg,"{'id': 2602, 'name': 'Scream Collection', 'pos...",35000000,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",https://www.screammovie.com,934433,tt17663992,en,Scream VI,...,2023-03-08,168541093,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,New York. New rules.,Scream VI,False,7.336,1039
5,False,/20uHjZ4yCPD2x0ndcxZaxM7hLIy.jpg,"{'id': 106498, 'name': 'Dungeons & Dragons Col...",151000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",https://www.dungeonsanddragons.movie,493529,tt2906216,en,Dungeons & Dragons: Honor Among Thieves,...,2023-03-23,202484920,134,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,No experience necessary.,Dungeons & Dragons: Honor Among Thieves,False,7.500,980
6,False,/h8gHn0OzBoaefsYseUByqsmEDMY.jpg,"{'id': 404609, 'name': 'John Wick Collection',...",90000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",https://johnwick.movie,603692,tt10366206,en,John Wick: Chapter 4,...,2023-03-22,406056266,170,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,No way back. One way out.,John Wick: Chapter 4,False,7.926,1341
7,False,/7bWxAsNPv9CXHOhZbJVlj2KxgfP.jpg,None,15000000,"[{'id': 27, 'name': 'Horror'}]",https://www.evildeadrisemovie.com,713704,tt13345606,en,Evil Dead Rise,...,2023-04-12,92071490,96,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Mommy loves you to death.,Evil Dead Rise,False,6.995,405
8,False,/a2tys4sD7xzVaogPntGsT1ypVoT.jpg,None,32500000,"[{'id': 53, 'name': 'Thriller'}, {'id': 35, 'n...",https://www.cocainebear.movie/,804150,tt14209916,en,Cocaine Bear,...,2023-02-22,89020092,96,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Get in line.,Cocaine Bear,False,6.395,979
9,False,/eEF40Xk2twM3WjRNZftfo771gjv.jpg,None,45000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.65.movie,700391,tt12261776,en,65,...,2023-03-02,55000000,93,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"65 million years ago, prehistoric Earth had a ...",65,False,6.275,961


In [14]:
new_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  50 non-null     bool   
 1   backdrop_path          50 non-null     object 
 2   belongs_to_collection  10 non-null     object 
 3   budget                 50 non-null     int64  
 4   genres                 50 non-null     object 
 5   homepage               50 non-null     object 
 6   id                     50 non-null     int64  
 7   imdb_id                49 non-null     object 
 8   original_language      50 non-null     object 
 9   original_title         50 non-null     object 
 10  overview               50 non-null     object 
 11  popularity             50 non-null     float64
 12  poster_path            50 non-null     object 
 13  production_companies   50 non-null     object 
 14  production_countries   50 non-null     object 
 15  release_

### Clean up the new movies data

In [15]:
import imdb
ia = imdb.IMDb()

def get_imdb_movie_info(imdb_id):
    # Search for the movie by ID
    movie = ia.get_movie(imdb_id)
    # Retrieve all available information
    info_dict = movie.data
    return info_dict

movie_info = get_imdb_movie_info('0113041')
movie_info_json = json_normalize(movie_info)

movie_info

{'localized title': 'Father of the Bride Part II',
 'cast': [<Person id:0000188[http] name:_Steve Martin_>,
  <Person id:0000473[http] name:_Diane Keaton_>,
  <Person id:0001737[http] name:_Martin Short_>,
  <Person id:0931090[http] name:_Kimberly Williams-Paisley_>,
  <Person id:0627624[http] name:_George Newbern_>,
  <Person id:0001085[http] name:_Kieran Culkin_>,
  <Person id:0000703[http] name:_BD Wong_>,
  <Person id:0324533[http] name:_Peter Michael Goetz_>,
  <Person id:0570033[http] name:_Kate McGregor-Stewart_>,
  <Person id:0011038[http] name:_Jane Adams_>,
  <Person id:0506405[http] name:_Eugene Levy_>,
  <Person id:0150427[http] name:_Rebecca Chambers_>,
  <Person id:0651074[http] name:_April Ortiz_>,
  <Person id:0736851[http] name:_Dulcy Rogers_>,
  <Person id:0030926[http] name:_Kathy Anthony_>,
  <Person id:0134852[http] name:_Adrian Canzoneri_>,
  <Person id:0015935[http] name:_Lori Alan_>,
  <Person id:0589353[http] name:_Stephanie Miller_>,
  <Person id:0583654[http]

#### Testing the Functions

In [16]:
box_office = movie_info.get('director')
test = movie_info.get('runtimes')
test

['106']

In [17]:
results = ia.search_movie('Avater')
# Get the first result
movie = results[0]
movie

<Movie id:1630029[http] title:_Avatar: The Way of Water (None)_>

#### Create Function to fill the unknowns of new data

In [18]:
import pandas as pd
import re

# Loop through each row in the DataFrame
def fill_unknown_rows(new_movies_df):
    if 'director' not in new_movies_df.columns:
        new_movies_df['director'] = None
    if 'cast' not in new_movies_df.columns:
        new_movies_df['cast'] = None

    # Create a list to store the rows that need to be updated
    rows_to_update = []

    for i, row in new_movies_df.iterrows():
        # Check if any values are missing or if budget/revenue is zero
        if row.isnull().any() or row['budget'] == 0 or row['revenue'] == 0:
            # If so, get the IMDB info for that movie
            try:
              imdb_id = re.sub('^tt', '', row['imdb_id']) # Remove 'tt' prefix
              imdb_info = get_imdb_movie_info(imdb_id)
              # Append the row index and IMDB info to the list
              rows_to_update.append((i, imdb_info))
            except:
              pass

    # Update the DataFrame with the new info for the selected rows
    for row_index, imdb_info in rows_to_update:
        row = new_movies_df.loc[row_index]
        actor_list = imdb_info.get('cast')
        actor_names = [person['name'] for person in actor_list[:5]]

        new_movies_df.at[row_index, 'original_title'] = imdb_info.get('original title')
        new_movies_df.at[row_index, 'budget'] = imdb_info.get('box office', {}).get('Budget')
        new_movies_df.at[row_index, 'revenue'] = imdb_info.get('box office', {}).get('Cumulative Worldwide Gross')
        new_movies_df.at[row_index, 'director'] = imdb_info.get('director', [{}])[0].get('name')
        new_movies_df.at[row_index, 'rating'] = imdb_info.get('rating')
        new_movies_df.at[row_index, 'genres'] = imdb_info.get('genres')[0]
        new_movies_df.at[row_index, 'cast'] = actor_names
        new_movies_df.at[row_index, 'vote_count'] = imdb_info.get('votes')
        new_movies_df.at[row_index, 'runtime'] = imdb_info.get('runtime')

    return new_movies_df


In [19]:
new_movies_df.isnull().sum()

adult                     0
backdrop_path             0
belongs_to_collection    40
budget                    0
genres                    0
homepage                  0
id                        0
imdb_id                   1
original_language         0
original_title            0
overview                  0
popularity                0
poster_path               0
production_companies      0
production_countries      0
release_date              0
revenue                   0
runtime                   0
spoken_languages          0
status                    0
tagline                   0
title                     0
video                     0
vote_average              0
vote_count                0
dtype: int64

In [20]:
# Fill out the unknowns if there are any
new_movies_df_filled=fill_unknown_rows(new_movies_df)
new_movies_df_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  50 non-null     bool   
 1   backdrop_path          50 non-null     object 
 2   belongs_to_collection  10 non-null     object 
 3   budget                 20 non-null     object 
 4   genres                 50 non-null     object 
 5   homepage               50 non-null     object 
 6   id                     50 non-null     int64  
 7   imdb_id                49 non-null     object 
 8   original_language      50 non-null     object 
 9   original_title         50 non-null     object 
 10  overview               50 non-null     object 
 11  popularity             50 non-null     float64
 12  poster_path            50 non-null     object 
 13  production_companies   50 non-null     object 
 14  production_countries   50 non-null     object 
 15  release_

In [21]:
new_movies_df_filled

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,director,cast,rating
0,False,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,None,"$18,000,000 (estimated)",Horror,https://www.thepopes-exorcist.movie,758323,tt13375076,en,The Pope's Exorcist,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Inspired by the actual files of Father Gabriel...,The Pope's Exorcist,False,7.409,21287,Julius Avery,"[Russell Crowe, Daniel Zovatto, Alex Essoe, Fr...",6.1
1,False,/iJQIbOPm81fPEGKt5BPuZmfnA54.jpg,None,"$100,000,000 (estimated)",Animation,https://www.thesupermariobros.movie,502356,tt6718170,en,The Super Mario Bros. Movie,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,The Super Mario Bros. Movie,False,7.498,92875,Aaron Horvath,"[Kevin Michael Richardson, Jack Black, Khary P...",7.3
2,False,/nDxJJyA5giRhXx96q1sWbOUjMBI.jpg,"{'id': 724848, 'name': 'Shazam! Collection', '...","$125,000,000 (estimated)",Action,,594767,tt10151854,en,Shazam! Fury of the Gods,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Oh. My. Gods.,Shazam! Fury of the Gods,False,6.784,64136,David F. Sandberg,"[Zachary Levi, Asher Angel, Jack Dylan Grazer,...",6.1
3,False,/A7JQ7MIV5fkIxceI5hizRIe6DRJ.jpg,"{'id': 284433, 'name': 'Guardians of the Galax...","$250,000,000 (estimated)",Action,https://www.marvel.com/movies/guardians-of-the...,447365,tt6791350,en,Guardians of the Galaxy Volume 3,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Once more with feeling.,Guardians of the Galaxy Volume 3,False,8.300,66244,James Gunn,"[Chukwudi Iwuji, Bradley Cooper, Pom Klementie...",8.4
4,False,/44immBwzhDVyjn87b3x3l9mlhAD.jpg,"{'id': 2602, 'name': 'Scream Collection', 'pos...","$35,000,000 (estimated)",Horror,https://www.screammovie.com,934433,tt17663992,en,Scream VI,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,New York. New rules.,Scream VI,False,7.336,65689,Matt Bettinelli-Olpin,"[Courteney Cox, Melissa Barrera, Jenna Ortega,...",6.7
5,False,/20uHjZ4yCPD2x0ndcxZaxM7hLIy.jpg,"{'id': 106498, 'name': 'Dungeons & Dragons Col...","$150,000,000 (estimated)",Action,https://www.dungeonsanddragons.movie,493529,tt2906216,en,Dungeons & Dragons: Honor Among Thieves,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,No experience necessary.,Dungeons & Dragons: Honor Among Thieves,False,7.500,77578,John Francis Daley,"[Chris Pine, Michelle Rodriguez, Regé-Jean Pag...",7.4
6,False,/h8gHn0OzBoaefsYseUByqsmEDMY.jpg,"{'id': 404609, 'name': 'John Wick Collection',...","$90,000,000 (estimated)",Action,https://johnwick.movie,603692,tt10366206,en,John Wick: Chapter 4,...,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,No way back. One way out.,John Wick: Chapter 4,False,7.926,142856,Chad Stahelski,"[Keanu Reeves, Laurence Fishburne, George Geor...",8.2
7,False,/7bWxAsNPv9CXHOhZbJVlj2KxgfP.jpg,None,"$12,000,000 (estimated)",Horror,https://www.evildeadrisemovie.com,713704,tt13345606,en,Evil Dead Rise,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Mommy loves you to death.,Evil Dead Rise,False,6.995,35935,Lee Cronin,"[Mirabai Pease, Richard Crouchley, Anna-Maree ...",7.1
8,False,/a2tys4sD7xzVaogPntGsT1ypVoT.jpg,None,"$35,000,000 (estimated)",Comedy,https://www.cocainebear.movie/,804150,tt14209916,en,Cocaine Bear,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Get in line.,Cocaine Bear,False,6.395,65483,Elizabeth Banks,"[Keri Russell, Alden Ehrenreich, O'Shea Jackso...",6.0
9,False,/eEF40Xk2twM3WjRNZftfo771gjv.jpg,None,"$45,000,000 (estimated)",Action,https://www.65.movie,700391,tt12261776,en,65,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"65 million years ago, prehistoric Earth had a ...",65,False,6.275,40251,Scott Beck,"[Adam Driver, Ariana Greenblatt, Chloe Coleman...",5.4


### Merge the old datasets with the new datasets
### Perform final clean up 

In [22]:
merged_df_with_valid_box_office['New_status']='False'
new_movies_df_filled['New_status']='True'
new_movies_df_filled['id'] = new_movies_df_filled['id'].astype(str)
# Check if ID already exists in merged_df_with_valid_box_office
existing_ids = merged_df_with_valid_box_office['id'].tolist()
new_movies_df_filled = new_movies_df_filled[~new_movies_df_filled['id'].isin(existing_ids)]

# Append new_movies_df_filled to merged_df_with_valid_box_office
combined_df_with_valid_box_office = pd.concat([merged_df_with_valid_box_office, new_movies_df_filled], ignore_index=True)
combined_df_with_valid_box_office.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17748 entries, 0 to 17747
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  10243 non-null  object 
 1   belongs_to_collection  1857 non-null   object 
 2   homepage               2835 non-null   object 
 3   id                     17748 non-null  object 
 4   imdb_id                10242 non-null  object 
 5   production_countries   10243 non-null  object 
 6   spoken_languages       10243 non-null  object 
 7   title                  10243 non-null  object 
 8   video                  10243 non-null  object 
 9   keywords               14022 non-null  object 
 10  backdrop_path          15139 non-null  object 
 11  recommendations        9988 non-null   object 
 12  budget                 17724 non-null  object 
 13  genres                 17180 non-null  object 
 14  original_language      17748 non-null  object 
 15  or

C:\Users\hongj\AppData\Local\Temp\ipykernel_13884\3312621062.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_with_valid_box_office['New_status']='False'


In [23]:
import ast

# Get top casts out of casts list
combined_df_with_valid_box_office['Top Cast'] = ''

for index, row in combined_df_with_valid_box_office.iterrows():
    try:
        cast_names = row['cast'].split('-')
    except:
        cast_names = row['cast']
    if cast_names is not None and isinstance(cast_names, list) and len(cast_names) > 0:
        top_cast_names = cast_names[:5]
        combined_df_with_valid_box_office.at[index, 'Top Cast'] = top_cast_names
        


In [24]:

def extract_genres(genres):
    if isinstance(genres, str):
        try:
            genres_list = ast.literal_eval(genres)
            return [genre['name'] for genre in genres_list]
        except (SyntaxError, ValueError):
            return genres.split('-')
    else:
        return []

merged_df_new_data = combined_df_with_valid_box_office.copy()
merged_df_new_data.genres = merged_df_new_data.genres.apply(extract_genres)
merged_df_new_data.genres.head()


0    [Animation, Adventure, Family, Comedy]
1              [Adventure, Fantasy, Family]
2                         [Romance, Comedy]
3                  [Comedy, Drama, Romance]
4                          [Comedy, Family]
Name: genres, dtype: object

In [25]:
merged_df_new_data["original_title"].fillna(merged_df_new_data["title"], inplace=True)
merged_df_new_data.drop("title", axis=1, inplace=True)
merged_df_new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17748 entries, 0 to 17747
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  10243 non-null  object 
 1   belongs_to_collection  1857 non-null   object 
 2   homepage               2835 non-null   object 
 3   id                     17748 non-null  object 
 4   imdb_id                10242 non-null  object 
 5   production_countries   10243 non-null  object 
 6   spoken_languages       10243 non-null  object 
 7   video                  10243 non-null  object 
 8   keywords               14022 non-null  object 
 9   backdrop_path          15139 non-null  object 
 10  recommendations        9988 non-null   object 
 11  budget                 17724 non-null  object 
 12  genres                 17748 non-null  object 
 13  original_language      17748 non-null  object 
 14  original_title         17748 non-null  object 
 15  ov

In [26]:
# Filter old and new data based on 'New_status'
merged_df_old = merged_df_new_data[merged_df_new_data['New_status'] == 'False']
merged_df_new = merged_df_new_data[merged_df_new_data['New_status'] == 'True']

In [27]:
# For local testing
# merged_df_new_data.to_csv('data_output.csv', index=False)

## Write Data to the database

In [28]:
!curl ipecho.net/plain

173.3.126.36


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    12  100    12    0     0     96      0 --:--:-- --:--:-- --:--:--    97


In [29]:
import numpy as np
import pandas as pd
import json
from sqlalchemy import create_engine
import psycopg2

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host="34.30.45.126",
    port=5432,
    database="finalproject",
    user="postgres",
    password="teamrocket"
)

# Create a cursor
cur = conn.cursor()

# Create a SQLAlchemy engine
engine = create_engine('postgresql+psycopg2://', creator=lambda: conn)


# cur.execute("DROP TABLE IF EXISTS movie_data;")

In [31]:
# Function to convert values to JSON based on their types
def convert_to_json(value):
    if isinstance(value, (np.integer, int, float, bool)):
        return json.dumps(value)
    elif isinstance(value, (list, dict)):
        return json.dumps(value)
    else:
        return value

# Save the DataFrame to the PostgreSQL database
table_name = "movie_data"

# Check if the table exists
cur.execute("SELECT EXISTS(SELECT 1 FROM information_schema.tables WHERE table_name = %s)", (table_name,))
table_exists = cur.fetchone()[0]

if table_exists:
    # Append new data to existing table
    # Convert values to JSON
    converted_new = merged_df_new.applymap(convert_to_json)

    # Append new data to existing table
    with engine.connect() as conn, conn.begin():
        converted_new.to_sql(
            table_name,
            conn,
            if_exists='append',
            index=False,
            method='multi'
        )
else:
    # Create table with old data
    # Convert values to JSON
    converted_old = merged_df_old.applymap(convert_to_json)
    # Create table with primary key and constraints
    create_table_query = f"""
        CREATE TABLE {table_name} (
            movie_key SERIAL PRIMARY KEY,
            {', '.join([f"{col} VARCHAR" for col in converted_old.columns if col != 'movie_key'])}
        )
    """
    engine.execute(create_table_query)

    # Insert old data into table
    with engine.connect() as conn, conn.begin():
        converted_old.to_sql(
            table_name,
            conn,
            if_exists='replace',
            index=False,
            method='multi'
        )

    # Append new data to table
    converted_new = merged_df_new.applymap(convert_to_json)

    with engine.connect() as conn, conn.begin():
        converted_new.to_sql(
            table_name,
            conn,
            if_exists='append',
            index=False,
            method='multi'
        )

In [34]:
# Execute a SELECT query to retrieve data from the table
cur.execute("SELECT * FROM movie_data")

# Fetch the top 10 rows from the result set
rows = cur.fetchmany(100)

# Get the column names from the description of the cursor
columns = [desc[0] for desc in cur.description]

# Output the column names
print(columns)

# Output the contents of the table (top 10 rows)
for row in rows:
    print(row)

['adult', 'belongs_to_collection', 'homepage', 'id', 'imdb_id', 'production_countries', 'spoken_languages', 'video', 'keywords', 'backdrop_path', 'recommendations', 'budget', 'genres', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'vote_average', 'vote_count', 'cast', 'director', 'New_status', 'rating', 'Top Cast']
('False', "{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}", 'http://toystory.disney.com/toy-story', '862', 'tt0114709', "[{'iso_3166_1': 'US', 'name': 'United States of America'}]", "[{'iso_639_1': 'en', 'name': 'English'}]", 'false', 'martial arts-jealousy-friendship-bullying-elementary school-friends-rivalry-rescue-mission-buddy-walkie talkie-toy car-boy next door-new toy-neighborhood-toy comes to life-resourcefulness', '/lxD5ak7BOoinRNehOCA85CQ8ubr.jpg', '

In [33]:
# # Close the cursor and connection
# cur.close()
# conn.close()